In [ ]:
import json
import copy
from ipyleaflet import (
    Map,
    GeoJSON,
    Layer
)

In [ ]:
m = Map(center=[53.88, 27.45], zoom=3, name='Base map')
m

In [ ]:
# geojson layer with hover handler
with open('./europe_110.geo.json') as f:
    data = json.load(f)

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.2
    }

selected_set = set()
selected_layer = None

def convert_selected_set_to_geojson(selected_set):
    geojson = {'type': 'FeatureCollection', 'features': []}
    geojson['features'] = [feature for feature in data['features'] if feature['properties']['iso_a3'] in selected_set]
    for feature in data['features']:
        feature['properties']['style'] = {'color': 'yellow', 'weight': 2,'fillColor': 'grey', 'fillOpacity': 0.2} 
    return geojson

def selected_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties['iso_a3']
    selected_set.remove(cid)
    if selected_layer is not None:
        m.remove_layer(selected_layer)
    selected_layer = GeoJSON(data=convert_selected_set_to_geojson(selected_set),
                             name='Selected EU Countries',
                             hover_style={'fillColor': 'yellow'}
                            )
    selected_layer.on_click(selected_onclick_handler)
    m.add_layer(selected_layer)

def geojson_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties['iso_a3']
    selected_set.add(cid)
    if selected_layer is not None:
        m.remove_layer(selected_layer)
    selected_layer = GeoJSON(data=convert_selected_set_to_geojson(selected_set),
                             name='Selected EU Countries',
                             hover_style={'fillColor': 'yellow'}
                            )
    selected_layer.on_click(selected_onclick_handler)
    m.add_layer(selected_layer)
    
    
geojson_layer = GeoJSON(data = data, name='EU Countries', hover_style={'fillColor': 'red'})
geojson_layer.on_click(geojson_onclick_handler)
m.add_layer(geojson_layer)